In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pickle
import math
import copy
import random
import time
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [ ]:
data = None
with open('datacompiled.p', 'rb') as f:
    data = pickle.load(f)

In [ ]:
PolFull = []
AziFull = []
FitFull = []
EpsFull = []
for nanoparticle in data:
    FitFull.append(float(nanoparticle[0]))
    for ligand in nanoparticle[1]:
        PolFull.append(ligand[0])
        AziFull.append(ligand[1])
        EpsFull.append(ligand[2])
PolMax = max(PolFull)
PolMin = min(PolFull)
AziMax = max(AziFull)
AziMin = min(AziFull)
FitMax = max(FitFull)
FitMin = min(FitFull)
EpsMax = max(EpsFull)
EpsMin = min(EpsFull)
discreteBlocks = 8
discreteBlocksFit = 10
PolStep = (PolMax-PolMin)/discreteBlocks
AziStep = (AziMax-AziMin)/discreteBlocks
FitStep = (FitMax-FitMin)/discreteBlocksFit
#x = Pol, y=Azi
def getIndex(Pol, Azi, PolMin, AziMin, PolStep, AziStep, discreteBlocks):    
    if Pol > 6.258: ###!!!! HCAK: to stop border values getting pushed over for now :(
        xGrid = discreteBlocks - 1
    else:
        xGrid = math.floor((Pol - PolMin)/PolStep) #0 indexed grid
    if Azi > 6.258: ###!!!! HCAK: to stop border values getting pushed over for now :(
        yGrid = discreteBlocks - 1
    else:
        yGrid = math.floor((Azi - AziMin)/AziStep)    
    #print('x:{}, y:{}'.format(xGrid, yGrid))
    index = (yGrid*discreteBlocks + xGrid)
    return index
def roundDownFit(Fit, FitMin, FitStep, discreteBlocks):
    return (math.floor((Fit - FitMin)/FitStep))*FitStep  
def roundDownEps(Eps):
    return float(int(Eps))

In [ ]:
inputFull = []
outputFull = []
for nanoparticle in data:
    inputFull.append(nanoparticle[1])
    outputFull.append(float(nanoparticle[0]))      

In [ ]:
outputDiscrete = []
for output_i in outputFull:
    #outputDiscrete.append(roundDownFit(output_i, FitMin, FitStep, discreteBlocksFit))
    outputDiscrete.append(output_i)

In [ ]:
plt.plot(outputFull, '.', label='full')
plt.plot(outputDiscrete, '.',label='Discretised')
plt.legend()
plt.title('Fit Discretised')

In [ ]:
inputDiscrete = []
for ligands in inputFull:
    inputDiscreteBlock = [0]*discreteBlocks**2
    for ligand in ligands:        
        updateIndex = getIndex(ligand[0],ligand[1], PolMin, AziMin, PolStep, AziStep, discreteBlocks)
        discreteEps = roundDownEps(ligand[2])   
        #print('adding eps {} at index {} for pol {} and azi {}'.format(discreteEps, updateIndex, ligand[0], ligand[1]))        
        inputDiscreteBlock[updateIndex] += discreteEps
    inputDiscrete.append(inputDiscreteBlock)    

In [ ]:
#plt.plot(inputDiscrete[0], '.')
#print((inputFull[0]))

In [ ]:
def customSample(data):
    random.shuffle(data)
    return data.pop()

In [ ]:
print(len(inputDiscrete))
print(len(outputDiscrete))

In [ ]:
validation_size = 100
validation_input = []
validation_output = []
training_input = []
training_output = []
for i in range(validation_size):
    validation_input.append(customSample(inputDiscrete))        
    validation_output.append(customSample(outputDiscrete))
training_input = inputDiscrete
training_output = outputDiscrete

In [ ]:
validation_input=np.array(validation_input)
print('validation input shape:{}'.format(np.shape(validation_input)))
validation_output=(np.array(validation_output)).reshape(np.shape(validation_output)[0],1)
print('validation output shape:{}'.format(np.shape(validation_output)))
training_input=np.array(training_input)
print('training input shape:{}'.format(np.shape(training_input)))
training_output=(np.array(training_output)).reshape(np.shape(training_output)[0],1)
print('training output shape:{}'.format(np.shape(training_output)))

In [ ]:
def random_sample(sample_length, list1, list2):
    len_list1 = np.shape(list1)[0]
    len_list2 = np.shape(list2)[0]
    if len_list1 != len_list2:
        return False
    else:
        sample_idx = random.sample(range(len_list1), sample_length)        
        s_list1 = [list1[i] for i in sample_idx]
        s_list2 = [list2[i] for i in sample_idx]
        return s_list1, s_list2

In [ ]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

input_count = 64
inputs = tf.placeholder(tf.float32, shape=[None, input_count])

output_count = 1
desired_outputs = tf.placeholder(tf.float32, shape=[None, output_count])

In [ ]:
#hidden layer 1
weights_mean_HL1 = 0
weights_mean_OUT = 0

weights_mean_HL2 = 0
weights_mean_OUT = 0

HL1_count = 25
HL2_count = 9

weights_HL1 = tf.Variable(tf.truncated_normal([input_count, HL1_count], weights_mean_HL1))
biases_HL1 = tf.Variable(tf.zeros([HL1_count]))
HL1_out = tf.nn.relu(tf.matmul(inputs, weights_HL1) + biases_HL1)

weights_HL2 = tf.Variable(tf.truncated_normal([HL1_count, HL2_count], weights_mean_HL2))
biases_HL2 = tf.Variable(tf.zeros([HL2_count]))
HL2_out = tf.nn.relu(tf.matmul(HL1_out, weights_HL2) + biases_HL2)

weights_out = tf.Variable(tf.truncated_normal([HL2_count, output_count], weights_mean_OUT))
biases_out = tf.Variable(tf.zeros([output_count]))
net_out = tf.nn.relu(tf.matmul(HL2_out, weights_out) + biases_out)

In [ ]:
error_function = tf.reduce_mean(tf.square(tf.subtract(net_out, desired_outputs)))
#train_step = tf.train.GradientDescentOptimizer(0.0001).minimize(error_function)
train_step = tf.train.AdamOptimizer(0.001).minimize(error_function)

In [ ]:
tf.global_variables_initializer().run()

In [ ]:
# restorer = tf.train.Saver()
# restorer.restore(sess, './tfmodel-0')

In [ ]:
track_loss = []
train_length = 100000
sample_length = 2000

start_train_sess = time.clock()
for i in range(train_length):
    sample_inputs, sample_outputs = random_sample(sample_length, training_input, training_output)
    _, loss = sess.run([train_step, error_function], 
                       feed_dict={inputs: np.array(sample_inputs), desired_outputs: np.array(sample_outputs)})

    if i%(train_length/10) == 0:
        print ('training step: {0}, loss:{1}'.format(i, loss))        

    track_loss.append(loss)

print('training session:{0}s for {1} steps'.format((time.clock() - start_train_sess), train_length))

plt.plot(track_loss, 'o')    
print('final loss:{0}'.format(track_loss[-1]))

In [ ]:
net_predict = sess.run(net_out, feed_dict={inputs: np.array(validation_input)})

In [ ]:
plt.plot(net_predict, '.', label='predicted')
plt.plot(validation_output, '.', label='ground truth')
plt.legend()
plt.xlabel('arb nanoparticle index')
plt.ylabel('fitness')
plt.show()

tolerance = FitStep
withinTol = 0
difference = []
for predict_i, truth_i in zip(net_predict, validation_output):
    difference.append(np.abs(predict_i - truth_i))
    if np.abs(predict_i - truth_i) <= tolerance:
        withinTol += 1
print('{}/{} within tolerance of {}'.format(withinTol, len(net_predict), tolerance))
plt.plot(difference, '.')

In [ ]:
# plt.plot(validation_output, '.', label='validation set', alpha=1)
# plt.plot(training_output, '.',label='training set', alpha=0.5)
# plt.legend()
# plt.xlabel('arb nanoparticle index')
# plt.ylabel('fitness')
# plt.xlim(0,100)

In [ ]:
# weights_HL1_save = sess.run(weights_HL1, feed_dict={inputs: np.array(training_inputs)})
# biases_HL1_save = sess.run(biases_HL1, feed_dict={inputs: np.array(training_inputs)})
# weights_out_save = sess.run(weights_out, feed_dict={inputs: np.array(training_inputs)})
# biases_out_save = sess.run(biases_out, feed_dict={inputs: np.array(training_inputs)})

# pickle.dump (weights_HL1_save, open("./tfmodel-0-weights_HL1.pickle", "wb"))
# pickle.dump (biases_HL1_save, open("./tfmodel-0-biases_HL1.pickle", "wb"))
# pickle.dump (weights_out_save, open("./tfmodel-0-weights_out.pickle", "wb"))
# pickle.dump (biases_out_save, open("./tfmodel-0-biases_out.pickle", "wb"))

In [ ]:
saver = tf.train.Saver(tf.global_variables())
saver.save(sess, './tfmodel-hl3-adam', global_step = 2)